In [2]:
import redis
from imdb import IMDb

In [ ]:
r = redis.StrictRedis(host='localhost', port=6379, db=0)
pipe = r.pipeline()

In [ ]:
from imdbpie import Imdb
imdb1 = Imdb()
imdb1 = Imdb(anonymize=True) # to proxy requests

# Creating an instance with caching enabled
# Note that the cached responses expire every 2 hours or so.
# The API response itself dictates the expiry time)
imdb1 = Imdb(cache=True)

In [ ]:
top250 = imdb1.top_250()
dict_top250 = {}
for i in range(len(top250)):
    dict_top250[((top250[i]['tconst'].encode('utf-8'))[2:len(top250[i]['tconst'].encode('utf-8'))])] = (top250[i]['title'].encode('utf-8'))

In [ ]:
# for movId, title in dict_top250.iteritems():
#     imdb2 = IMDb()
#     my = imdb2.get_movie(movId)
#     for castMember in my['cast'][0:10]:
#         full_person = imdb2.get_person(castMember.getID(), info=["filmography"])
#         pipe.execute()
#         try:
#             r[castMember['name'].encode('utf-8')]
#         except KeyError: 
#             try:
#                 pipe.set(castMember.getID(), (castMember['name'].encode('utf-8'), full_person["actress"]))
#             except:
#                 pipe.set(castMember.getID(), (castMember['name'].encode('utf-8'), full_person["actor"]))

cast_dict = {}

for movId, title in dict_top250.iteritems():
    cast_list = []
#     print movId
    imdb2 = IMDb()
    my = imdb2.get_movie(movId)
#     pipe.execute()
    for castMember in my['cast'][0:10]:
#         cast_dict[castMember['name'].encode('utf-8')] = castMember.getID();
            cast_list.append(castMember.getID());
#     print cast_list
#     pipe.set(movId, cast_list);        
    cast_dict[movId] = cast_list
print cast_dict

In [ ]:
def test(movies):
    """input: list of 250 movies
        output: hashtable"""
    act_assoc_map = {}
    for movie in movies:
        l_act = movies.get(movie)
        for act in l_act:
            for act_id in range(len(l_act)):
                if act not in act_assoc_map.keys():
                    if l_act[act_id] != act:
                        act_assoc_map[act] = [l_act[act_id]]
                else:
                    if l_act[act_id] != act:
                        act_assoc_map[act].append(l_act[act_id])
    for actor in act_assoc_map.keys():
        act_assoc_map[actor] = list(set(act_assoc_map[actor]))
    return act_assoc_map

hashtable = test(cast_dict)
#print hashtable
for key, value in hashtable.iteritems():
    print str(value)
    pipe.execute();
    for i in value:
        pipe.lpush(key, i)

In [ ]:
len(hashtable.keys())

In [ ]:
for key, value in hashtable.iteritems():
    print len(value)

In [ ]:
full_person = imdb2.get_person('0815495')

In [ ]:
full_person